In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [2]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/plbart_base_embeddings/processed_embeddings/plbart_embed_line_by_line_avg.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,0.728083,-2.532038,0.368371,-2.176581,2.044621,-1.313796,-1.297907,-0.010166,-2.046683,-1.360018,...,1.297135,1.766562,-1.514855,-3.513744,-3.533521,1.863348,-0.564521,1,12150,critical
1,0.685935,-1.755593,0.898242,-2.131532,1.925080,-0.883668,0.744284,-1.481731,-0.992622,-3.059729,...,1.514646,1.104696,-1.490182,-3.218683,-2.581105,2.782470,1.722228,1,10168,critical
2,2.014241,-1.051502,-0.134887,-1.408881,2.449127,-1.582936,-1.319389,-0.076099,-1.271313,-1.734699,...,1.010146,0.754675,-1.005410,-2.877481,-2.725385,2.086285,0.148330,1,12783,critical
3,0.581555,-1.134039,-0.140375,-2.288345,2.719025,-1.412528,-1.934057,-0.066457,-2.365500,-1.635481,...,1.017341,0.646247,-0.976179,-2.642240,-2.973845,2.562416,-0.896226,1,12754,critical
4,1.624544,-1.399879,0.244179,-1.355493,2.514415,-1.868643,-1.279655,0.199190,-1.480868,-1.251517,...,1.078729,0.707406,-1.293822,-3.561596,-2.398942,1.914786,0.081015,1,4277,critical


In [3]:
# Separate features and labels
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [4]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")


Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.8310
F1_macro: 0.5757
Accuracy: 0.8310
Precision_macro: 0.5978
Recall_macro: 0.5673

Bagging Metrics:
F1_micro: 0.8451
F1_macro: 0.5378
Accuracy: 0.8451
Precision_macro: 0.5861
Recall_macro: 0.5360

XGBoost Metrics:
F1_micro: 0.8348
F1_macro: 0.5847
Accuracy: 0.8348
Precision_macro: 0.6080
Recall_macro: 0.5754


In [5]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")



Final Test Results:

Random Forest Metrics:
F1_micro: 0.8207
F1_macro: 0.5701
Precision_macro: 0.5819
Recall_macro: 0.5637
Accuracy: 0.8207
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       786
           1       0.28      0.21      0.24       123

    accuracy                           0.82       909
   macro avg       0.58      0.56      0.57       909
weighted avg       0.80      0.82      0.81       909


Bagging Metrics:
F1_micro: 0.8493
F1_macro: 0.5531
Precision_macro: 0.6119
Recall_macro: 0.5460
Accuracy: 0.8493
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.96      0.92       786
           1       0.35      0.13      0.19       123

    accuracy                           0.85       909
   macro avg       0.61      0.55      0.55       909
weighted avg       0.80      0.85      0.82       909


XGBoost Metrics:
F1_micro: 0.8251
F1_macro: 

In [7]:
# Save results to a pickle file
output_path = "/Users/mac/Desktop/Code_Smell_Detection/feature_envy/results/plbart/result_plbart_avg.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")



Results saved to /Users/mac/Desktop/Code_Smell_Detection/feature_envy/results/plbart/result_plbart_avg.pkl
